<a href="https://colab.research.google.com/github/amal-gafoor/license-plate-detection/blob/main/yolov8n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# model building

In [ ]:
import os
os.chdir('/content/drive/MyDrive/yolov8n')

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="nl32Kt3DDnnXJHJUrZoL")
project = rf.workspace("mochoye").project("license-plate-detector-ogxxg")
version = project.version(2)
dataset = version.download("yolov8")


In [ ]:
dir_path='/content/drive/MyDrive/yolov8n/License-Plate-Detector-2'

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLOv8n model
model = YOLO("yolov8n.yaml")

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data=os.path.join(dir_path,"converted.yaml"), epochs=100)


In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/yolov8n/runs/detect/train2/weights/best.pt','/content/drive/MyDrive/yolov8n/License-Plate-Detector-2/yolov8n_model.pt')

# prediction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install ultralytics
!pip install easyocr

In [ ]:
from ultralytics import YOLO
model=YOLO('/content/drive/MyDrive/yolov8n/License-Plate-Detector-2/yolov8n_model.pt')

In [ ]:
# libreries
import cv2
import easyocr
from google.colab.patches import cv2_imshow

# initializing reader
reader=easyocr.Reader(['en'])

# write directions
img_save_path='/content/drive/MyDrive/yolov8n/License-Plate-Detector-2/pred image/img1.jpg'
vid_save_path='/content/drive/MyDrive/yolov8n/License-Plate-Detector-2/pred video/vid1.mp4'

# image detection
def img_detection(img_path):
  img=cv2.imread(img_path)
  pred_img=model.predict(img)[0]

  # drawing bountion boxes
  for box in pred_img.boxes:
    x1,y1,x2,y2=map(int,box.xyxy[0])
    crop=img[y1:y2,x1:x2]

    ocr_result=reader.readtext(crop)
    if ocr_result:
      text=ocr_result[0][-2] # [([cardinates],text,confidence)]
      cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
      w,h=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0] #[0] because (textsize,baseline) format
      rect_x1 = x1
      rect_y1 = y1 - h - 10  # 10px padding above box
      rect_x2 = x1 + w + 6    # 6px padding on sides
      rect_y2 = y1

      if rect_x1 < 0:
          rect_x1 = 0

      if rect_y1 < 0:
          rect_y1 = 0

      cv2.rectangle(img,(rect_x1,rect_y1),(rect_x2,rect_y2),(0,255,0),-1)
      cv2.putText(img,text,(rect_x1+3,y1-5),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
  # cv2_imshow(img)
  # cv2.waitKey(0)
  # cv2.destroyAllWindows()
  cv2.imwrite(img_save_path,img)
  return

# video detection
def vid_detection(vid_path):
  cap=cv2.VideoCapture(vid_path)
  frame_width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps=int(cap.get(cv2.CAP_PROP_FPS))
  fourcc=cv2.VideoWriter_fourcc(*'mp4v')
  out=cv2.VideoWriter(vid_save_path,fourcc,fps,(frame_width,frame_height))

  while True:
    ret,frame=cap.read()
    if not ret:
      break
    pred_img=model.predict(frame)[0]
    for box in pred_img.boxes:
      x1,y1,x2,y2=map(int,box.xyxy[0])
      crop=frame[y1:y2,x1:x2]

      ocr_result=reader.readtext(crop)
      if ocr_result:
        text=ocr_result[0][-2] # [([cardinates],text,confidence)]
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
        w,h=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0]
        rect_x1 = x1
        rect_y1 = y1 - h - 10
        rect_x2 = x1 + w + 6
        rect_y2 = y1

        if rect_x1<0:
          rect_x1=0

        if rect_y1<0:
          rect_y1=0

        cv2.rectangle(frame,(rect_x1,rect_y1),(rect_x2,rect_y2),(0,255,0),-1)
        cv2.putText(frame,text,(rect_x1+3,y1-5),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
    # cv2_imshow(frame)
    out.write(frame)
  cap.release()
  # cap.destroyAllWindows()
  return

while True:
  choice=input('img or vid: ')
  if choice=='img':
    img_input_path=input('enter image path: ')
    img_detection(img_input_path)
    break
  elif choice=='vid':
    vid_input_path=input('enter video path: ')
    vid_detection(vid_input_path)
    break
  else:
    print('invalid input')

